<a href="https://colab.research.google.com/github/JulioLaz/clasificacion_textos/blob/main/Clasificaci%C3%B3n_de_texto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Configurar ambiente

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
import torch
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.3 MB/s eta 0:00:00


In [3]:
!python -m spacy download es_core_news_md
import spacy

nlp = spacy.load('es_core_news_md')

2023-09-22 18:19:36.740057: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 12.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')


In [4]:
!pip install jellyfish
import jellyfish

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.1 MB/s eta 0:00:00


In [5]:
import re, os, random, pickle

In [6]:
import pandas as pd

ruta_archivo = "/content/drive/MyDrive/Chatbot/df_dialogo.csv"

df_dialogo = pd.read_csv(ruta_archivo)

# 2. Normalizando dialogos

In [7]:
#Función para encontrar la raiz de las palabras
def raiz(palabra):
    palabra_encontrada = palabra
    max_similitud = 0.0

    for verbo in lista_verbos:
        similitud = jellyfish.jaro_similarity(palabra, verbo)
        if similitud > max_similitud:
            max_similitud = similitud
            palabra_encontrada = verbo

    if max_similitud >= 0.93:
        return palabra_encontrada, max_similitud
    else:
        return palabra, max_similitud

In [8]:
def reemplazar_terminacion(frase):
    terminaciones = ["es", "me", "as", "ste", "te"]
    palabras = frase.split()

    for i, palabra in enumerate(palabras):
        for terminacion in terminaciones:
            if palabra.endswith(terminacion):
                palabras[i] = palabra[:-len(terminacion)] + "r"
                break

    nueva_frase = " ".join(palabras)
    return nueva_frase

In [9]:
def tratamiento_texto(texto):
  trans = str.maketrans('áéíóú','aeiou')
  texto = texto.lower()
  texto = texto.translate(trans)
  # texto = re.sub(r"[^\w\s]", '', texto)
  texto = re.sub(r"[^\w\s+\-*/]", '', texto)

  texto = " ".join(texto.split())
  return texto

frase='QUÉ DÍA ES HOY,    JULIO ALBERTO?'
texto_tratado=tratamiento_texto(frase)
print(texto_tratado)

que dia es hoy julio alberto


In [10]:
import pickle

archivo_pickle_verbos = "/content/drive/MyDrive/Chatbot/verbos/lista_verbos.pickle"
archivo_pickle_verbos_irregulares = "/content/drive/MyDrive/Chatbot/verbos/verbos_irregulares.pickle"

with open(archivo_pickle_verbos, "rb") as archivo:
        lista_verbos = pickle.load(archivo)

with open(archivo_pickle_verbos_irregulares, "rb") as archivo_irr:
        lista_verbos_irregulares = pickle.load(archivo_irr)


print(lista_verbos)
print('---------------------------------')
print(lista_verbos_irregulares)

['parar', 'recomendar', 'cancelar', 'fanatizar', 'amaran o amasen', 'exponer', 'obedecer', 'quejar', 'echar', 'legitimar', 'perjudicar', 'organizar', 'molar', 'objetar', 'considerar', 'golear', 'mover', 'acertar', 'reunir', 'regir', 'ilusionar', 'simpatizar', 'conjeturar', 'helar', 'quitar', 'amariamos', 'destacar', 'llegar', 'sincronizar', 'lesionar', 'seducir', 'asistir', 'conservar', 'acordar', 'salvar', 'relucir', 'graduar', 'forzar', 'dar', 'deplorar', 'batear', 'mofar', 'estropear', 'aplastar', 'wasapeo', 'gestionar', 'suprimir', 'gruñir', 'progresar', 'suscribir', 'noticiar', 'cavar', 'alejar', 'galopar', 'virar', 'medir', 'actualizar', 'humanizar', 'convivir', 'gratificar', 'digerir', 'tocar', 'zonificar', 'amariais', 'aterrizar', 'hojear', 'cometer', 'sufrir', 'reciclar', 'obturar', 'divertir', 'ondear', 'listar', 'determinar', 'alentar', 'sumar', 'reflexionar', 'ames', 'anotar', 'mitificar', 'escribir', 'ilustrar', 'obtener', 'subir', 'socorrer', 'desprender', 'agregar', 'gan

In [11]:
#Función para devolver los tokens normalizados del texto
def normalizar(texto):
  tokens=[]
  doc = nlp(texto)
  for t in doc:
    lemma=lista_verbos_irregulares.get(t.text, t.lemma_.split()[0])
    lemma=re.sub(r'[^\w\s+\-*/]', '', lemma)
    if t.pos_ in ('VERB','PROPN','PRON','NOUN','AUX','SCONJ','ADJ','ADV','NUM') or lemma in lista_verbos:
      if t.pos_=='VERB':
        lemma = reemplazar_terminacion(lemma)
        tokens.append(raiz(tratamiento_texto(lemma)))
      else:
        tokens.append(tratamiento_texto(lemma))

  tokens = list(dict.fromkeys(tokens))
  tokens = list(filter(None, tokens))
  cadena_texto = ' '.join(str(x) for x in tokens)
  return cadena_texto
  # return tokens

In [12]:
normalizar("texto es muy largo")

'texto ser mucho largo'

In [13]:
#Normalizando las frases
label_encoder = LabelEncoder()
df_dialogo['palabras'] = df_dialogo['dialogo'].apply(lambda x: normalizar(x))
# df_dialogo['palabras'] = df_dialogo['dialogo'].apply(lambda x: ' '.join(normalizar(x)))
df_dialogo['tipo_num'] = label_encoder.fit_transform(df_dialogo['tipo'])
df_dialogo = df_dialogo[df_dialogo.palabras.values!='']
df_dialogo

,dialogo,respuesta,tipo,interseccion,jaro_winkler,probabilidad,similarity,palabras,tipo_num
0,que puedo hacer en,"Pues, ¡diviertete!",Funcion,1.000000,0.905556,1.000000,0.776515,"que poder ('hacer', 1.0)",8
1,que puedo hacer en,"Pues, ¡divertirse! ¿No?",Funcion,1.000000,0.905556,1.000000,0.776515,"que poder ('hacer', 1.0)",8
2,que sabes hacer,"Puedo responder muchas preguntas, sé todo sobr...",Funcion,0.666667,0.893636,0.893636,0.503103,"que ('saber', 1.0) ('hacer', 1.0)",8
3,que sabes hacer,"Puedo ayudarte con análisis de datos, aprender...",Funcion,0.666667,0.893636,0.893636,0.503103,"que ('saber', 1.0) ('hacer', 1.0)",8
4,que puedes hacer,"Puedo responder muchas preguntas, sé todo sobr...",Funcion,0.666667,0.884470,0.884470,0.503103,"que poder ('hacer', 1.0)",8
...,...,...,...,...,...,...,...,...,...
1145,okay,Me alegra haber podido ayudarte.,Agradecimiento,0.000000,0.000000,0.000000,0.000000,okay,0
1147,chatgpt,ChatGPT es una plataforma de procesamiento de ...,Identidad,0.000000,0.000000,0.000000,0.000000,"('chatgpt', 0.7142857142857143)",9
1148,ola,¡Hola!,Saludos,0.000000,0.000000,0.000000,0.000000,ola,13
1149,ola,"Hola, ¿cómo puedo ayudar?",Saludos,0.000000,0.000000,0.000000,0.000000,ola,13


In [14]:
# Imprimir diccionario
relacion_diccionario = {}

# Iterar sobre las filas del DataFrame
for tipo, tipo_num in zip(df_dialogo['tipo'], df_dialogo['tipo_num']):
    relacion_diccionario[tipo_num] = tipo

# Imprimir el diccionario
print(relacion_diccionario)

{8: 'Funcion', 12: 'Otros', 5: 'Edad', 14: 'Sentimiento', 9: 'Identidad', 11: 'Origen', 13: 'Saludos', 10: 'Nombre', 6: 'ElProfeAlejo', 0: 'Agradecimiento', 4: 'Despedida', 2: 'Contacto', 7: 'Error', 15: 'Usuario', 1: 'Aprendizaje', 3: 'Continuacion'}


# 3. Entrenando con Naive Bayes

In [15]:
# Separar los datos en características (X) y etiquetas (y)
X = df_dialogo['palabras']
y = df_dialogo['tipo_num']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorizar los datos de texto
vectorizer = CountVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Entrenar el clasificador de Naive Bayes
modelo_NB = MultinomialNB()
modelo_NB.fit(X_train_vect, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = modelo_NB.predict(X_test_vect)

# Calcular el accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Precisión:", accuracy)

Precisión: 0.6739130434782609


In [16]:
# Guardar el vectorizador y el modelo en un archivo
with open('/content/drive/MyDrive/Chatbot/modelo_naive_bayes.pickle', 'wb') as archivo:
    pickle.dump((vectorizer, modelo_NB), archivo)

# Cargar el vectorizador y el modelo desde el archivo
with open('/content/drive/MyDrive/Chatbot/modelo_naive_bayes.pickle', 'rb') as archivo:
    vectorizer, modelo_NB = pickle.load(archivo)

In [17]:
# Calcular la precisión por clase
unique_classes = df_dialogo['tipo_num'].unique()
for cls in unique_classes:
    cls_indices = y_test == cls
    cls_accuracy = accuracy_score(y_test[cls_indices], y_pred[cls_indices])
    print("Accuracy para la clase", df_dialogo[df_dialogo.tipo_num == cls]['tipo'].unique()[0], ":", cls_accuracy)

Accuracy para la clase Funcion : 0.45454545454545453
Accuracy para la clase Otros : 0.8970588235294118
Accuracy para la clase Edad : 0.3333333333333333
Accuracy para la clase Sentimiento : 0.26666666666666666
Accuracy para la clase Identidad : 0.7647058823529411
Accuracy para la clase Origen : 0.625
Accuracy para la clase Saludos : 0.9032258064516129
Accuracy para la clase Nombre : 0.25
Accuracy para la clase ElProfeAlejo : 0.7142857142857143
Accuracy para la clase Agradecimiento : 0.4375
Accuracy para la clase Despedida : 0.7647058823529411
Accuracy para la clase Contacto : 0.2
Accuracy para la clase Error : 0.25
Accuracy para la clase Usuario : 0.7142857142857143
Accuracy para la clase Aprendizaje : 0.375
Accuracy para la clase Continuacion : 0.16666666666666666


In [18]:
# Procesando la nueva frase
frase = ' '.join(normalizar('como haces para aprender tan rapido?'))
nueva_frase_vect = vectorizer.transform([frase])

# Realizar la predicción
prediccion = modelo_NB.predict(nueva_frase_vect)

diccionario = {14: 'Sentimiento', 13: 'Saludos', 10: 'Nombre', 9: 'Identidad', 6: 'ElProfeAlejo', 1: 'Aprendizaje', 8: 'Funcion', 15: 'Usuario', 11: 'Origen', 5: 'Edad', 0: 'Agradecimiento', 3: 'Continuacion', 2: 'Contacto', 4: 'Despedida', 12: 'Otros', 7: 'Error'}
llave_buscada = prediccion[0]
clase_encontrada = diccionario[llave_buscada]

print("La frase", frase, "se clasifica como: ", clase_encontrada)

La frase c o m o   ( ' h a c e r ' ,   1 . 0 )   ( ' a p r e n d e r ' ,   1 . 0 )   t a n t o   r a p i d o se clasifica como:  Otros


# 4. Entrenando con Random Forest

In [19]:
# Separar los datos en características (X) y etiquetas (y)
X = df_dialogo['palabras']
y = df_dialogo['tipo_num']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorizar los datos de texto
vectorizer = CountVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Entrenar el clasificador Random Forest
Modelo_RF = RandomForestClassifier()
Modelo_RF.fit(X_train_vect, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = Modelo_RF.predict(X_test_vect)

# Calcular el accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Precisión:", accuracy)

Precisión: 0.7347826086956522


In [20]:
# Guardar el vectorizador y el modelo en un archivo
with open('/content/drive/MyDrive/Chatbot/modelo_random_forest.pickle', 'wb') as archivo:
    pickle.dump((vectorizer, Modelo_RF), archivo)

# Cargar el vectorizador y el modelo desde el archivo
with open('/content/drive/MyDrive/Chatbot/modelo_random_forest.pickle', 'rb') as archivo:
    vectorizer, Modelo_RF = pickle.load(archivo)

In [21]:
# Calcular la precisión por clase
unique_classes = df_dialogo['tipo_num'].unique()
for cls in unique_classes:
    cls_indices = y_test == cls
    cls_accuracy = accuracy_score(y_test[cls_indices], y_pred[cls_indices])
    print("Accuracy para la clase", df_dialogo[df_dialogo.tipo_num == cls]['tipo'].unique()[0], ":", cls_accuracy)

Accuracy para la clase Funcion : 0.6363636363636364
Accuracy para la clase Otros : 0.8529411764705882
Accuracy para la clase Edad : 0.5
Accuracy para la clase Sentimiento : 0.4
Accuracy para la clase Identidad : 0.7647058823529411
Accuracy para la clase Origen : 0.75
Accuracy para la clase Saludos : 0.8709677419354839
Accuracy para la clase Nombre : 0.75
Accuracy para la clase ElProfeAlejo : 0.7142857142857143
Accuracy para la clase Agradecimiento : 0.625
Accuracy para la clase Despedida : 0.8823529411764706
Accuracy para la clase Contacto : 0.6
Accuracy para la clase Error : 0.75
Accuracy para la clase Usuario : 0.8571428571428571
Accuracy para la clase Aprendizaje : 0.375
Accuracy para la clase Continuacion : 0.16666666666666666


In [22]:
# Procesando la nueva frase
frase = ' '.join(normalizar('como haces para aprender tan rapido?'))
nueva_frase_vect = vectorizer.transform([frase])

# Realizar la predicción
prediccion = Modelo_RF.predict(nueva_frase_vect)

diccionario = {14: 'Sentimiento', 13: 'Saludos', 10: 'Nombre', 9: 'Identidad', 6: 'ElProfeAlejo', 1: 'Aprendizaje', 8: 'Funcion', 15: 'Usuario', 11: 'Origen', 5: 'Edad', 0: 'Agradecimiento', 3: 'Continuacion', 2: 'Contacto', 4: 'Despedida', 12: 'Otros', 7: 'Error'}
llave_buscada = prediccion[0]
clase_encontrada = diccionario[llave_buscada]

print("La frase", frase, "se clasifica como: ", clase_encontrada)

La frase c o m o   ( ' h a c e r ' ,   1 . 0 )   ( ' a p r e n d e r ' ,   1 . 0 )   t a n t o   r a p i d o se clasifica como:  Otros


# 5. Entrenando con Transformers

In [ ]:
# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
df_train, df_test = train_test_split(df_dialogo, test_size=0.2, random_state=42)

# Cargar el modelo preentrenado de BERT para clasificación en español
model_name = 'dccuchile/bert-base-spanish-wwm-uncased'
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=df_dialogo['tipo_num'].nunique())
tokenizer = BertTokenizer.from_pretrained(model_name)

# Tokenizar y codificar las frases de entrenamiento
train_inputs = tokenizer.batch_encode_plus(
    df_train['palabras'].tolist(),
    max_length=128,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

# Tokenizar y codificar las frases de prueba
test_inputs = tokenizer.batch_encode_plus(
    df_test['palabras'].tolist(),
    max_length=128,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

# Preparar los datos de entrenamiento y prueba
train_data = torch.utils.data.TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], torch.tensor(df_train['tipo_num'].tolist()))
test_data = torch.utils.data.TensorDataset(test_inputs['input_ids'], test_inputs['attention_mask'], torch.tensor(df_test['tipo_num'].tolist()))

# Definir el optimizador y la función de pérdida
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Entrenamiento del modelo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()

train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True)

for epoch in range(5):  # Número de épocas de entrenamiento
    total_loss = 0

    for batch in train_dataloader:
        input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    print("Epoch:", epoch + 1, "Loss:", total_loss)

# Evaluación del modelo
model.eval()
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)

with torch.no_grad():
    predictions = []
    true_labels = []

    for batch in test_dataloader:
        input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)

        outputs = model(input_ids, attention_mask=attention_mask)

        _, predicted_labels = torch.max(outputs.logits, dim=1)

        predictions.extend(predicted_labels.tolist())
        true_labels.extend(labels.tolist())

accuracy = accuracy_score(true_labels, predictions)
print("Precisión:", accuracy)

In [ ]:
# Guardar el modelo entrenado
# ruta_modelo = '/content/drive/MyDrive/Chatbot/'
# model.save_pretrained(ruta_modelo)
# tokenizer.save_pretrained(ruta_modelo)

In [25]:
#Cargar el modelo entrenado
ruta_modelo = '/content/drive/MyDrive/Chatbot/'
Modelo_TF = BertForSequenceClassification.from_pretrained(ruta_modelo)
tokenizer_TF = BertTokenizer.from_pretrained(ruta_modelo)

In [26]:
# Calcular la precisión por clase
unique_classes = df_dialogo['tipo_num'].unique()

for class_label in unique_classes:
    # Filtrar los datos por clase
    class_data = df_dialogo[df_dialogo['tipo_num'] == class_label]

    # Preparar los datos de la clase para evaluar
    tokens = tokenizer_TF.batch_encode_plus(
        class_data['palabras'].tolist(),
        truncation=True,
        padding=True,
        return_tensors='pt'
    )

    inputs = tokens['input_ids']
    attention_mask = tokens['attention_mask']
    labels = class_data['tipo_num'].tolist()

    # Pasar los datos de la clase por el modelo
    with torch.no_grad():
        outputs = Modelo_TF(inputs, attention_mask=attention_mask)

    predicted_labels = outputs.logits.argmax(dim=1).tolist()

    # Calcular la precisión para la clase
    accuracy = accuracy_score(labels, predicted_labels)
    print(f"Precisión por clase {df_dialogo[df_dialogo.tipo_num == class_label]['tipo'].unique()[0]}: {accuracy}")

Precisión por clase Funcion: 0.8904109589041096
Precisión por clase Otros: 0.963076923076923
Precisión por clase Edad: 0.6774193548387096
Precisión por clase Sentimiento: 0.9154929577464789
Precisión por clase Identidad: 0.8888888888888888
Precisión por clase Origen: 0.82
Precisión por clase Saludos: 0.9833333333333333
Precisión por clase Nombre: 0.75
Precisión por clase ElProfeAlejo: 0.8888888888888888
Precisión por clase Agradecimiento: 0.9242424242424242
Precisión por clase Despedida: 0.9444444444444444
Precisión por clase Contacto: 0.5172413793103449
Precisión por clase Error: 0.875
Precisión por clase Usuario: 0.8648648648648649
Precisión por clase Aprendizaje: 0.7674418604651163
Precisión por clase Continuacion: 0.7419354838709677


In [27]:
# Procesar nueva frase
frase = ' '.join(normalizar('donde vives?'))

# Tokenizar la frase de entrada
tokens = tokenizer_TF.encode_plus(
    frase,
    add_special_tokens=True,
    max_length=128,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

# Obtener los input_ids y attention_mask
input_ids = tokens['input_ids']
attention_mask = tokens['attention_mask']

# Realizar la predicción
with torch.no_grad():
    outputs = Modelo_TF(input_ids, attention_mask)

# Obtener las etiquetas predichas
etiquetas_predichas = torch.argmax(outputs.logits, dim=1)

# Decodificar las etiquetas predichas
etiquetas_decodificadas = etiquetas_predichas.tolist()

diccionario = {14: 'Sentimiento', 13: 'Saludos', 10: 'Nombre', 9: 'Identidad', 6: 'ElProfeAlejo', 1: 'Aprendizaje', 8: 'Funcion', 15: 'Usuario', 11: 'Origen', 5: 'Edad', 0: 'Agradecimiento', 3: 'Continuacion', 2: 'Contacto', 4: 'Despedida', 12: 'Otros', 7: 'Error'}
llave_buscada = etiquetas_decodificadas[0]
clase_encontrada = diccionario[llave_buscada]
print("La frase", frase, "se clasifica como: ", clase_encontrada)

La frase d o n d e   ( ' v i v i r ' ,   1 . 0 ) se clasifica como:  Otros
